# EX03 Solutions
Solutions to the third week's exercises on stereo vision and triangulation.

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt

### Exercise 3.1
We project the world point $Q$ into each camera using the pinhole model
$\mathbf{q} \sim K [R \mid t] Q$ where $Q$ is expressed in homogeneous coordinates.
The intrinsics and extrinsics are specified in the exercise.

In [2]:
# Camera intrinsics
K = np.array([[1000,0,300],
              [0,1000,200],
              [0,0,1]])

# Camera extrinsics
t1 = np.zeros(3)
R1 = np.eye(3)

R2 = Rotation.from_euler('xyz',[0.7,-0.5,0.8]).as_matrix()
t2 = np.array([0.2,2.0,1.0])

# Homogeneous point
Q = np.array([1,0.5,4,1])

# Projection helper
def project(K,R,t,Q):
    P = K @ np.hstack([R, t.reshape(3,1)])
    q = P @ Q
    return q[:2]/q[2]

q1 = project(K,R1,t1,Q)
q2 = project(K,R2,t2,Q)
print('q1 =', q1)
print('q2 =', q2)


q1 = [550. 325.]
q2 = [582.47256835 185.98985776]


### Exercise 3.2
The cross product matrix $[p]_	imes$ satisfies $[p]_	imes q = p 	imes q$.


In [3]:
def CrossOp(p):
    x,y,z = p
    return np.array([[0,-z,y],
                     [z,0,-x],
                     [-y,x,0]])

# verification on random vectors
p1 = np.random.randn(3)
p2 = np.random.randn(3)
left = CrossOp(p1) @ p2
right = np.cross(p1,p2)
print('difference:', np.linalg.norm(left-right))


difference: 6.938893903907228e-18


### Exercise 3.3
The essential matrix is $E=[t]_	imes R$ and the fundamental matrix is
$F = K^{-T} E K^{-1}$.

In [4]:
E = CrossOp(t2) @ R2
K_inv = np.linalg.inv(K)
F = K_inv.T @ E @ K_inv
print('F =')
print(F)


F =
[[ 3.29311881e-07  8.19396327e-07  1.79162592e-03]
 [ 5.15532551e-07 -8.76915984e-07  9.31426656e-05]
 [-1.29882755e-03  1.51951700e-03 -1.10072682e+00]]


### Exercise 3.4
The epipolar line in the second image corresponding to $q_1$ is $l = F q_1$.


In [5]:
q1_h = np.array([*q1,1])
l = F @ q1_h
print('l =', l)


l = [ 2.23905126e-03  9.16878739e-05 -1.32123895e+00]


### Exercise 3.5
A point lies on a line if the inner product between the homogeneous point and line vanishes:
$q_2^T l = 0$.


In [6]:
q2_h = np.array([*q2,1])
val = q2_h @ l
print('q2^T l =', val)


q2^T l = 4.440892098500626e-16


### Exercise 3.6
We verify numerically that
$Q = egin{bmatrix}R_1^T & -R_1^T t_1 \ 0 & 1\end{bmatrix} 	ilde Q$
when $	ilde Q = egin{bmatrix}R_1 & t_1 \ 0 & 1\end{bmatrix} Q$.


In [7]:
T = np.block([[R1, t1.reshape(3,1)],
                   [np.zeros((1,3)), np.ones((1,1))]])
T_inv = np.block([[R1.T, -R1.T @ t1.reshape(3,1)],
                  [np.zeros((1,3)), np.ones((1,1))]])
Q_tilde = T @ Q
Q_recovered = T_inv @ Q_tilde
print('recovered Q equals original:', np.allclose(Q_recovered, Q))


recovered Q equals original: True


### Exercise 3.7
Using the relation above we project purely in the coordinate frame of camera~1.


In [8]:
R2_tilde = R2 @ R1.T
_t2 = t2 - R2 @ R1.T @ t1
q1_alt = K @ np.hstack([np.eye(3), np.zeros((3,1))]) @ Q_tilde
q1_alt = q1_alt[:2]/q1_alt[2]
q2_alt = K @ np.hstack([R2_tilde, _t2.reshape(3,1)]) @ Q_tilde
q2_alt = q2_alt[:2]/q2_alt[2]
print('q1 from camera frame:', q1_alt)
print('q2 from camera frame:', q2_alt)


q1 from camera frame: [550. 325.]
q2 from camera frame: [582.47256835 185.98985776]


### Exercise 3.8
We compute the fundamental matrix for the car stereo pair contained in
`TwoImageDataCar.npy`.


In [9]:
import os
if os.path.exists('TwoImageDataCar.npy'):
    data = np.load('TwoImageDataCar.npy', allow_pickle=True).item()
    pts1 = data['pts1']
    pts2 = data['pts2']
    # eight-point algorithm with normalization
    def normalize(pts):
        mean = pts.mean(axis=0)
        std = np.sqrt(2) / np.std(pts[:,0:2], axis=0).mean()
        T = np.array([[std,0,-std*mean[0]],[0,std,-std*mean[1]],[0,0,1]])
        pts_h = np.hstack([pts, np.ones((pts.shape[0],1))])
        pts_n = (T @ pts_h.T).T
        return pts_n, T
    pts1n, T1 = normalize(pts1)
    pts2n, T2 = normalize(pts2)
    A = np.stack([pts1n[:,0]*pts2n[:,0], pts1n[:,0]*pts2n[:,1], pts1n[:,0],
                  pts1n[:,1]*pts2n[:,0], pts1n[:,1]*pts2n[:,1], pts1n[:,1],
                  pts2n[:,0], pts2n[:,1], np.ones(len(pts1n))], axis=1)
    U,S,Vt = np.linalg.svd(A)
    F_n = Vt[-1].reshape(3,3)
    # enforce rank-2 constraint
    Uf,Sf,Vtf = np.linalg.svd(F_n)
    Sf[-1]=0
    F_n = Uf @ np.diag(Sf) @ Vtf
    F_car = T2.T @ F_n @ T1
    print('F_car =\n', F_car)
else:
    print('TwoImageDataCar.npy not found; skipping computation.')


TwoImageDataCar.npy not found; skipping computation.


### Exercise 3.9
The snippet below allows clicking in the first image and drawing the corresponding epipolar
line in the second image.


In [10]:
# This cell demonstrates how to interactively draw an epipolar line in image 2
# given a click in image 1. It requires an interactive backend.
# Uncomment the lines below to use in a local environment.
# if 'F_car' in globals():
#     img1, img2 = data['img1'], data['img2']
#     %matplotlib qt
#     fig, (ax1,ax2) = plt.subplots(1,2)
#     ax1.imshow(img1); ax2.imshow(img2)
#     ax1.set_title('Image 1'); ax2.set_title('Image 2')
#     plt.show()
#     def onclick(event):
#         if event.inaxes==ax1:
#             q = np.array([event.xdata, event.ydata,1])
#             l = F_car @ q
#             x = np.array([0, img2.shape[1]-1])
#             y = -(l[0]*x + l[2])/l[1]
#             ax2.plot(x,y,'r-')
#             fig.canvas.draw()
#     cid = fig.canvas.mpl_connect('button_press_event', onclick)
# else:
#     print('Fundamental matrix not available.')
print('Interactive demo not executed.')

Interactive demo not executed.


### Exercise 3.10
This cell performs the symmetric operation: clicking in image two draws the epipolar
line in image one using the transpose of the fundamental matrix.


In [11]:
# Symmetric interactive demo for clicking in image 2 and drawing
# the corresponding epipolar line in image 1. Requires an interactive backend.
# Uncomment for use in a local environment.
# if 'F_car' in globals():
#     img1, img2 = data['img1'], data['img2']
#     %matplotlib qt
#     fig, (ax1,ax2) = plt.subplots(1,2)
#     ax1.imshow(img1); ax2.imshow(img2)
#     ax1.set_title('Image 1'); ax2.set_title('Image 2')
#     plt.show()
#     def onclick(event):
#         if event.inaxes==ax2:
#             q = np.array([event.xdata, event.ydata,1])
#             l = F_car.T @ q
#             x = np.array([0, img1.shape[1]-1])
#             y = -(l[0]*x + l[2])/l[1]
#             ax1.plot(x,y,'g-')
#             fig.canvas.draw()
#     cid = fig.canvas.mpl_connect('button_press_event', onclick)
# else:
#     print('Fundamental matrix not available.')
print('Interactive demo not executed.')

Interactive demo not executed.


### Exercise 3.11
We implement linear triangulation for a point observed in multiple views.


In [12]:
def triangulate(qs, Ps):
    A = []
    for q, P in zip(qs, Ps):
        x,y = q
        A.append(x*P[2]-P[0])
        A.append(y*P[2]-P[1])
    A = np.stack(A)
    _,_,Vt = np.linalg.svd(A)
    X = Vt[-1]
    return X[:3]/X[3]

P1 = K @ np.hstack([R1, t1.reshape(3,1)])
P2 = K @ np.hstack([R2, t2.reshape(3,1)])
Q_est = triangulate([q1,q2],[P1,P2])
print('triangulated Q:', Q_est)
q1_proj = project(K,R1,t1,np.append(Q_est,1))
q2_proj = project(K,R2,t2,np.append(Q_est,1))
print('reprojection error cam1:', np.linalg.norm(q1_proj-q1))
print('reprojection error cam2:', np.linalg.norm(q2_proj-q2))


triangulated Q: [1.  0.5 4. ]
reprojection error cam1: 2.8421709430404007e-13
reprojection error cam2: 2.5421149729252077e-13
